Après plusieurs entretiens qui ont pour la majorité abouti à une mise en suspension du processsus de recrutement avec comme argument principale : 
"Nous sommes très intéressés par votre profil mais malheureusement nous avons peu de missions pour le moment, nous vous recontactons dès que nous avons quelcque chose et de votre côté tenez-nous au courant si vous avez des propositions."

Ayant atteint mon objectif personnel de renforcer mes compétences en python à un niveau proche de celui que j'ai en R (7 ans d'expérience) grâce notamment aux deux livres suivants:

1. Apprendre à programmer en python
2. Machine learning avec python

En attendant deux autres retours d'entretiens, j'ai acquis un troisième livre (Deep Learning avec TensorFlow) et je m'apprêtais poursuivre mon apprentissage du deep learning initié il y a plus d'un an à travers la spécialisation deep learning de *deepleraning.ai* sur coursera lorsque je me suis dit que je voulais aller plus loin que la simple lecture du livre et la résolution des exercices proposés.

Après quelques minutes de réflexion j'ai assez rapidement trouvé un sujet qui m'occuperait vraiment l'esprit et qui me ferait grandement progresser dans mon métier de data scientist. J'ai décidé de construire un modèle de machine translation pour traduire mon rapport de stage de fin d'étude.